# ModSSC | Inductive SSL

Run an inductive semi supervised method and evaluate it.

## Objective
- Show the minimal steps to run this component in a notebook setting.
- Provide the exact objects to look at (outputs, shapes, metrics) to confirm it worked.

## Prerequisites
- Python 3.11+.
- `pip install modssc`.
- Optional dependencies depend on datasets and backends. If an import fails, install the matching extra and rerun.

## Outline
1) Imports and configuration
2) Core run (the part that does the work)
3) Sanity checks and outputs



## Notebook notes

This notebook shows how to use the `modssc.inductive` brick on a toy dataset.

> **Feature-based** methods (tabular / embeddings).


## Imports and configuration



In [ ]:
import numpy as np

from modssc.inductive import DeviceSpec, InductiveDataset, get_method_class
from modssc.inductive.methods.self_training import SelfTrainingSpec

rng = np.random.default_rng(0)
X = rng.normal(size=(300, 16)).astype(np.float32)
y = (X[:, 0] + 0.25 * X[:, 1] > 0).astype(np.int64)

# 5 labels per class
labeled = np.zeros(X.shape[0], dtype=bool)
for c in np.unique(y):
    idx = np.flatnonzero(y == c)
    labeled[rng.choice(idx, size=5, replace=False)] = True

X_l = X[labeled]
y_l = y[labeled]
X_u = X[~labeled]

spec = SelfTrainingSpec(
    classifier_id="knn",
    classifier_backend="numpy",
    classifier_params={"k": 7},
    confidence_threshold=0.8,
    max_iter=10,
)
method_cls = get_method_class("self_training")
method = method_cls(spec=spec)
data = InductiveDataset(X_l=X_l, y_l=y_l, X_u=X_u)
method.fit(data, device=DeviceSpec(device="cpu"), seed=0)
pred = method.predict(X)
acc = float((pred == y).mean())
acc

## Inspect available methods

In [ ]:
from modssc.inductive import available_methods, get_method_info

available_methods()

In [ ]:
get_method_info("self_training")

## Outputs

- The last cells should print key shapes and a minimal metric or artifact summary.
- If something fails early, the error should point to a missing optional dependency.


## Next steps
- Explore the adjacent notebooks in this folder for the other pipeline components.
- If you hit an optional dependency error, install the suggested extra and rerun.
